In [2]:
import numpy as np

In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(output):
    return output * (1 - output)

def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def maxpool2x2(x):
    n, h, w = x.shape
    pooled = np.zeros((n, h // 2, w // 2))
    for i in range(0, h, 2):
        for j in range(0, w, 2):
            pooled[:, i // 2, j // 2] = np.max(x[:, i:i+2, j:j+2], axis=(1, 2))
    return pooled

def maxpool2x2_backward(dout, x):
    n, h, w = x.shape
    dx = np.zeros_like(x)
    for i in range(0, h, 2):
        for j in range(0, w, 2):
            window = x[:, i:i+2, j:j+2]
            max_val = np.max(window, axis=(1, 2), keepdims=True)
            mask = (window == max_val)
            dx[:, i:i+2, j:j+2] += mask * dout[:, i//2, j//2][:, None, None]
    return dx

class Conv2D:
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = np.random.randn(num_filters, 1, filter_size, filter_size) * 0.1

    def forward(self, x):
        self.x = x
        n, h, w = x.shape
        f = self.filter_size
        out_h = h - f + 1
        out_w = w - f + 1
        output = np.zeros((self.num_filters, out_h, out_w))

        for idx, filt in enumerate(self.filters):
            for i in range(out_h):
                for j in range(out_w):
                    region = x[:, i:i+f, j:j+f]
                    output[idx, i, j] = np.sum(region * filt)

        return output

    def backward(self, d_out, learning_rate=0.01):
        d_filters = np.zeros_like(self.filters)
        f = self.filter_size

        for idx in range(self.num_filters):
            for i in range(d_out.shape[1]):
                for j in range(d_out.shape[2]):
                    region = self.x[:, i:i+f, j:j+f]
                    d_filters[idx] += np.sum(region * d_out[idx, i, j])

        self.filters -= learning_rate * d_filters

class SimpleCNN:
    def __init__(self):
        self.conv = Conv2D(num_filters=4, filter_size=3)
        self.fc_weights = np.random.randn(4 * 13 * 13, 1) * 0.1  # after conv + pool
        self.fc_bias = np.zeros((1,1))

    def forward(self, x):
        self.x = x
        self.conv_out = self.conv.forward(x)          # Conv
        self.relu_out = relu(self.conv_out)           # ReLU
        self.pool_out = maxpool2x2(self.relu_out)     # MaxPool
        self.flatten = self.pool_out.reshape(1, -1)   # Flatten
        self.output = sigmoid(self.flatten @ self.fc_weights + self.fc_bias)  # FC + Sigmoid
        return self.output

    def backward(self, y_true, learning_rate=0.01):
        # Loss: binary cross-entropy
        error = self.output - y_true  # (1,1)
        d_output = error * sigmoid_derivative(self.output)  # (1,1)

        # FC weights
        d_fc_weights = self.flatten.T @ d_output
        d_fc_bias = d_output

        # Backprop into flatten
        d_flatten = d_output @ self.fc_weights.T
        d_pool = d_flatten.reshape(self.pool_out.shape)

        # Backprop through maxpool
        d_relu = maxpool2x2_backward(d_pool, self.relu_out)

        # Backprop through ReLU
        d_conv = d_relu * relu_derivative(self.conv_out)

        # Backprop through conv
        self.conv.backward(d_conv, learning_rate)

        # Update FC
        self.fc_weights -= learning_rate * d_fc_weights
        self.fc_bias -= learning_rate * d_fc_bias

    def train(self, x, y, epochs=50):
        for epoch in range(epochs):
            output = self.forward(x)
            loss = - (y * np.log(output + 1e-8) + (1 - y) * np.log(1 - output + 1e-8))
            self.backward(y)
            if epoch % 5 == 0:
                print(f"Epoch {epoch}, Loss: {loss[0][0]:.4f}")

    def predict(self, x):
        return self.forward(x) > 0.5


In [10]:
if __name__ == "__main__":
    # Create a single grayscale image: shape (1, 28, 28)
    image = np.random.rand(1, 28, 28)
    label = np.array([[1]])  # Binary class label

    model = SimpleCNN()
    model.train(image, label, epochs=50)

    pred = model.predict(image)
    print("Predicted class:", int(pred[0][0]))


Epoch 0, Loss: 0.7371
Epoch 5, Loss: 0.6532
Epoch 10, Loss: 0.5124
Epoch 15, Loss: 0.3770
Epoch 20, Loss: 0.2714
Epoch 25, Loss: 0.2032
Epoch 30, Loss: 0.1610
Epoch 35, Loss: 0.1329
Epoch 40, Loss: 0.1137
Epoch 45, Loss: 0.0998
Predicted class: 1
